In [1]:
import pathlib
from dataclasses import dataclass

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interpn
from pydrad.parse import Strand
import sunpy.map
import fiasco
import eispac.core

import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentBase, InstrumentSDOAIA, InstrumentLOSVelocity, InstrumentHinodeXRT, ChannelBase
from synthesizAR.atomic import EmissionModel

## Interface for Loading Hydrodynamic Simulations

- Load a single HYDRAD simulation
- For each loop, select a random timestep within a given interval
- Interpolate given timestep to spatial domain of strand

In [2]:
class HydradSingleRunInterface:
    """
    This takes a single HYDRAD run and maps it to fieldlines in a model skeleton.
    A random timestep is sampled from the HYDRAD run
    """
    name = 'HYDRAD_single_run'

    def __init__(self, strand, time_range=None):
        self.strand = Strand(strand)
        self.time_range = self.strand.time[[0,-1]] if time_range is None else time_range
        self.time_indices = np.where(np.logical_and(self.strand.time>=self.time_range[0],
                                                    self.strand.time<=self.time_range[-1]))[0]

    def load_results(self, loop):
        # Choose snapshot from simulation
        snapshot = self.strand[np.random.choice(self.time_indices)]
        s_norm = (snapshot.coordinate / self.strand.loop_length).decompose()
        # Interpolate each quantity to loop coordinate
        quantities = []
        for name in ['electron_temperature', 'ion_temperature', 'electron_density', 'velocity']:
            q = getattr(snapshot, name)
            f_interp = interp1d(s_norm, q.value, fill_value='extrapolate')
            q_interp = f_interp(loop.field_aligned_coordinate_center_norm) * q.unit
            quantities.append(q_interp[np.newaxis,:])

        time = [0,] * u.s
        return [time,] + quantities

## Set up Instruments

In [3]:
# Pull this in to get right observer location
m_eis = sunpy.map.Map('../pipeline/data/EIS/level_2.5/eis_20120924_105026.fe_12_195_119.2c-0.int.fits')

In [4]:
pad_fov = [10,10] * u.arcsec

In [5]:
aia = InstrumentSDOAIA([0,1]*u.s, m_eis.observer_coordinate, pad_fov=pad_fov)

INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


In [6]:
xrt = InstrumentHinodeXRT([0,1]*u.s, m_eis.observer_coordinate, ['Al-poly'], pad_fov=pad_fov)

In [7]:
los_vel = InstrumentLOSVelocity([0,]*u.s, m_eis.observer_coordinate, pad_fov=pad_fov )

## Instrument for Spectral Imaging

In [8]:
#density_grid = np.logspace(7,12,30) * u.Unit('cm-3')
#temperature_grid = 10**np.arange(5,7.5,0.05) * u.K
#emission_model = EmissionModel(
#    density_grid,
#    fiasco.Ion('Fe XII', temperature_grid),
#    fiasco.Ion('Fe XIII', temperature_grid),
#)
#emission_model.calculate_emissivity_table('model_data/emissivity_table.zarr')
#emission_model.to_asdf('model_data/emission_model.asdf')

# If emissivity table is already calculated, just load it here
emission_model = EmissionModel.from_asdf('model_data/emission_model.asdf')

In [16]:
los_vel.name

'los_velocity'

In [9]:
@dataclass
class ChannelSpectralImager(ChannelBase):
    ion_name: str = None
    wavelength: u.Quantity = None
    psf_width: u.Quantity = (1,1)*u.pix

    def __post_init__(self):
        self.name = f'{self.ion_name} {self.wavelength.to_value("AA"):.3f}'


class InstrumentSpectralImager(InstrumentBase):
    name = 'Hinode_EIS'

    def __init__(self, observing_time, observer, line_list, **kwargs):
        self.channels = [ChannelSpectralImager(ion_name=name, wavelength=wave) for name, wave in line_list]
        super().__init__(observing_time, observer, **kwargs)

    @property
    def resolution(self):
        # Should be EIS resolution
        return u.Quantity([1.9968, 1], 'arcsec / pix')

    @property
    def observatory(self):
        return 'Hinode'

    @property
    def detector(self):
        return 'EIS'

    @property
    def telescope(self):
        return self.observatory

    def get_instrument_name(self, channel):
        return self.detector

    def get_header(self, channel, coordinates, **kwargs):
        header = super().get_header(channel, coordinates, **kwargs)
        header['line_id'] = channel.name
        header['measrmnt'] = 'intensity'
        return header

    @staticmethod
    def calculate_intensity_kernel(loop, channel, **kwargs):
        # Retrieve needed ion object
        emission_model = kwargs.get('emission_model')
        if emission_model is None:
            raise ValueError('Must pass emission model to .observe() method.')
        ion_mapping = {i.ion_name_roman: i for i in emission_model}
        ion = ion_mapping[channel.ion_name]

        # Select emissivity for relevant transition
        wave_grid, emiss_grid = emission_model.get_emissivity(ion)
        idx = np.argmin(np.fabs(wave_grid - channel.wavelength))
        wave = wave_grid[idx]
        emissivity = emiss_grid[...,idx]
        # Multiply ionization fraction by emissivity
        emissivity *= ion.ioneq[:,np.newaxis]
        
        # Interpolate emissivity to loop temperature and density
        T = loop.electron_temperature
        n = loop.density
        emissivity_interp = interpn(
            (emission_model.temperature.to(T.unit).value, emission_model.density.to(n.unit).value),
            emissivity.value,
            np.stack((T.value.flatten(), n.value.flatten()), axis=1),
            method='linear',
            fill_value=None,
            bounds_error=False,
        )
        emissivity_interp = np.reshape(emissivity_interp, T.shape)
        emissivity_interp = u.Quantity(np.where(emissivity_interp < 0., 0., emissivity_interp), emissivity.unit)
        
        # Combine all pieces
        energy_per_photon = wave.to('erg', equivalencies=u.spectral()) / u.photon
        scalar_factor = 0.83 * ion.abundance * energy_per_photon / (4*np.pi*u.steradian)
        kernel = scalar_factor * n * emissivity_interp

        return (kernel.value, kernel.unit)

In [10]:
line_list = [
    ('Fe XII', 186.854*u.AA),
    ('Fe XII', 186.887*u.AA),
    ('Fe XII', 195.119*u.AA),
    ('Fe XIII', 203.795*u.AA),
    ('Fe XIII', 203.826*u.AA),
    ('Fe XIII', 202.044*u.AA),
]
eis = InstrumentSpectralImager([0,]*u.s, m_eis.observer_coordinate, line_list, pad_fov=pad_fov)

## Produce images for all instruments
- Loop over HYDRAD runs
- Load results to Skeleton
- Produce AIA, XRT, velocity, and EIS images for each model run

In [11]:
hydrad_runs = [
    pathlib.Path('model_data/hydrad_runs/run_1/'),
    pathlib.Path('model_data/hydrad_runs/run_2/'),
    pathlib.Path('model_data/hydrad_runs/run_3/'),
    pathlib.Path('model_data/hydrad_runs/run_4'),
    pathlib.Path('model_data/hydrad_runs/run_7/'),
]

First, load all results into loop skeleton

In [12]:
model_data_dir = pathlib.Path('model_data/')
skeletons = {}
for run in hydrad_runs:
    skeletons[run.name] = synthesizAR.Skeleton.from_asdf('model_data/base_skeleton.asdf')
    hydrad = HydradSingleRunInterface(run, time_range=[36,48]*u.h)
    skeletons[run.name].load_loop_simulations(hydrad)

Purposefully instantiating client here becuase it interferes with how the model is loaded in an annoying way if you're not writing to a Zarr file.

In [14]:
client = distributed.Client()

In [21]:
for run in hydrad_runs:
    for instr in [aia, xrt, eis, los_vel]:
        if instr.name == 'Hinode_EIS':
            sim_maps = instr.observe(skeletons[run.name], emission_model=emission_model)
        elif instr.name == 'los_velocity':
            sim_maps = instr.observe(skeletons[run.name], observer=los_vel.observer)
        else:
            sim_maps = instr.observe(skeletons[run.name],)
        res_dir = model_data_dir / instr.name / run.name
        res_dir.mkdir(parents=True, exist_ok=True)
        for k in sim_maps:
            for i,m in enumerate(sim_maps[k]):
                fname = res_dir / f'{"_".join(k.split())}_t{i}.fits'
                sim_maps[k][i].save(fname, overwrite=True)

/Users/wtbarnes/mambaforge/envs/hinode-2023-loops-poster/lib/python3.11/site-packages/scipy/interpolate/_fitpack_impl.py:194: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
/Users/wtbarnes/mambaforge/envs/hinode-2023-loops-poster/lib/python3.11/site-packages/scipy/interpolate/_fitpack_impl.py:194: RuntimeWarning: A theoretically impossible result when finding a smoothing spline
with fp = s. Probable cause: s too small. (abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ier][0]))
/Users/wtbarnes/mambaforge/envs/hinode-2023-loops-poster/lib/python3.11/site-packages/scipy/interpolate/_fitpack_impl.py:194: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
  warnings.warn(RuntimeWarning(_iermess[ie